In [ ]:
!pip install diffusers transformers accelerate torch safetensors

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import torch
from diffusers import StableDiffusionXLInpaintPipeline
from PIL import Image, ImageDraw
import numpy as np

# Load the SDXL Refiner model
pipe = StableDiffusionXLInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
)
pipe = pipe.to("cuda")

# Load and prepare the image
original_image = Image.open("/content/assignment.jpg").convert("RGB")
width, height = original_image.size
new_width, new_height = width + 256, height + 256

# Create a new blank image with expanded dimensions
new_image = Image.new("RGB", (new_width, new_height))
new_image.paste(original_image, (128, 128))

# Create a mask for the areas to be inpainted
mask = Image.new("RGB", (new_width, new_height), "white")
mask_draw = ImageDraw.Draw(mask)
mask_draw.rectangle((128, 128, width+128, height+128), fill="black")

# Convert to numpy arrays
image_array = np.array(new_image)
mask_array = np.array(mask)

# Perform outpainting in chunks
chunk_size = 512  # Adjust based on available memory
for y in range(0, new_height, chunk_size):
    for x in range(0, new_width, chunk_size):
        # Define the region to process
        region = (x, y, min(x+chunk_size, new_width), min(y+chunk_size, new_height))

        # Crop the image and mask to the current region
        region_image = Image.fromarray(image_array).crop(region)
        region_mask = Image.fromarray(mask_array).crop(region)

        # Ensure the cropped region is in RGB mode
        region_image = region_image.convert("RGB")
        region_mask = region_mask.convert("RGB")

        # Perform inpainting on the region
        with torch.no_grad():
            result = pipe(
                prompt="A natural extension of the original image, seamless blend, highly detailed",
                image=region_image,
                mask_image=region_mask,
                num_inference_steps=20,
                guidance_scale=7.5,
                strength=0.99
            ).images[0]

        # Ensure the result is in RGB mode and matches the region size
        result = result.convert("RGB")
        if result.size != (region[2] - region[0], region[3] - region[1]):
            result = result.resize((region[2] - region[0], region[3] - region[1]))

        # Paste the result back into the full image
        new_image.paste(result, region)

        # Clear CUDA cache after each chunk
        torch.cuda.empty_cache()

# Save the final result
new_image.save("/content/outpainted_image.png")


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]